In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,desc

In [42]:
spark = SparkSession.builder.master('local').appName('Viewers Turned Streamers').getOrCreate()

In [43]:
df_input = spark.read.options(header='True', InferSchema='True').csv('twitch_sessions.csv').orderBy('user_id', 'session_start')
#df_input.show()

In [44]:
##filtering out users whose first session where viewer
df_input.createOrReplaceTempView("fb")
df_viewer = spark.sql("select user_id,session_start,session_type,rnk from ( select user_id,session_start,session_type, dense_rank() over(partition by user_id order by user_id, session_start) as rnk from fb) a where a.rnk =1 and a.session_type='viewer'")
#df_viewer.show()

In [45]:
##finding out total streaming session from above users whose first activity was not viewer
df_input.createOrReplaceTempView("fb")
df_streamer = spark.sql("select user_id,session_start,session_type,rnk from ( select user_id,session_start,session_type, dense_rank() over(partition by user_id order by user_id, session_start) as rnk from fb) a where a.rnk > 1 and a.session_type!='viewer'")
df_streamer_count = df_streamer.groupBy('user_id').count().select(col('user_id'), col('count').alias('total_stream_count')).sort(desc('total_stream_count'))
df_streamer_count = df_streamer_count.withColumnRenamed("user_id","user_ids")
#df_streamer_count.show()

In [46]:
##finding out users who turned into streamer from viewer
df_output = df_viewer.join(df_streamer_count, df_viewer.user_id == df_streamer_count.user_ids , 'inner').orderBy('user_id').selectExpr('user_id','total_stream_count')
df_output.show()

+-------+------------------+
|user_id|total_stream_count|
+-------+------------------+
|      1|                 1|
|      3|                 1|
+-------+------------------+

